In [ ]:
import os
import pandas as pd

all_headlines=[]
for filename in os.listdir('drive/MyDrive/00AI/articles/'):
  if 'Articles' in filename:
    headlines_df=pd.read_csv('drive/MyDrive/00AI/articles/'+filename)
    all_headlines.extend(list(headlines_df.headline.values))
len(all_headlines)

9335

In [ ]:
all_headlines[21:30]

['Unknown',
 'Measuring the Damage of a False Narrative in a Refugee Camp',
 'A ‘Blue’ Florida? For Democrats, No Quick Demographic Fixes',
 'Unknown',
 'The Tet Offensive Was Just the Beginning',
 'Trump Has Got Democrats Right Where He Wants Them',
 'Heart: Flu Elevates Risk of Heart Attack',
 'Unknown',
 'Unknown']

首先清除"Unknown"資料

In [ ]:
all_headlines=[h for h in all_headlines if h !="Unknown"]
len(all_headlines)

8603

In [ ]:
all_headlines[20:30]

['Measuring the Damage of a False Narrative in a Refugee Camp',
 'A ‘Blue’ Florida? For Democrats, No Quick Demographic Fixes',
 'The Tet Offensive Was Just the Beginning',
 'Trump Has Got Democrats Right Where He Wants Them',
 'Heart: Flu Elevates Risk of Heart Attack',
 'Employer Health System: Flawed, but Innovative',
 'Alone With My Husband’s Secret',
 'In Talk to Republicans, President Puts Blame on Democrats',
 'Making Sense of the Jobs Report',
 'Homes for Sale in Brooklyn and Manhattan']

移除標點符號與變化成小寫, 再將資料及內出現每個字轉換成數值，標記化(Tokenization)

In [ ]:
import tensorflow as tf
tf.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters= '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}\t\n',
    lower=True,
    split=' ',
    char_level=False,
    oov_token=None,
    analyzer=None)

In [ ]:
#Tokenizer=tf.keras.preprocessing.text.Tokenizer
#tokenizer=Tokenizer()
tokenizer=tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(all_headlines)
total_words=len(tokenizer.word_index)+1
print('Total words',total_words)

Total words 11753


In [ ]:
all_headlines[:1]

['I Stand  With the ‘She-Devils’']

In [ ]:
#subset_dict={key: value for key, value in tokenizer.word_index.items() \
#       if key in ['a','man','a','plan','a','canal','panama']}
subset_dict={key: value for key, value in tokenizer.word_index.items() \
       if key in ['a','man','plan','canal','panama']}
print(subset_dict)

{'a': 2, 'plan': 82, 'man': 137, 'panama': 2823, 'canal': 9558}


In [ ]:
tokenizer.texts_to_sequences(['a','man','a','plan','a','canal','panama'])

[[2], [137], [2], [82], [2], [9558], [2823]]

In [ ]:
tokenizer.texts_to_sequences(['a','man','a','plan','a','canal','panama'])[1]

[137]

In [ ]:
input_sequences=[]
for line in all_headlines:
  token_list=tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
    partial_sequence = token_list[:i+1]
    input_sequences.append(partial_sequence)
print(tokenizer.sequences_to_texts(input_sequences[:5]))
input_sequences[:5]

['i stand', 'i stand with', 'i stand with the', 'i stand with the ‘she', 'i stand with the ‘she devils’']


[[30, 314],
 [30, 314, 11],
 [30, 314, 11, 1],
 [30, 314, 11, 1, 3395],
 [30, 314, 11, 1, 3395, 5242]]

為了讓序列長度相同 填充序列

In [ ]:
from keras.utils.data_utils import pad_sequences
import numpy as np
max_sequence_len=max([len(x) for x in input_sequences])
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))
input_sequences[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
        30, 314], dtype=int32)

In [ ]:
predictors = input_sequences[:,:-1]
labels=input_sequences[:,-1]
labels[:5]

array([ 314,   11,    1, 3395, 5242], dtype=int32)

In [ ]:
utils=tf.keras.utils
labels=utils.to_categorical(labels,num_classes=total_words)

In [ ]:
input_len=max_sequence_len-1
model = tf.keras.models.Sequential()
layers=tf.keras.layers
model.add(layers.Embedding(total_words,10,input_length=input_len))
model.add(layers.LSTM(100))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(total_words, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 27, 10)            117530    
                                                                 
 lstm_1 (LSTM)               (None, 100)               44400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 11753)             1187053   
                                                                 
Total params: 1,348,983
Trainable params: 1,348,983
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [ ]:
model.fit(predictors,labels,epochs=30,verbose=1)

Epoch 1/30
1666/1666 [==============================] - 25s 14ms/step - loss: 7.8872
Epoch 2/30
1666/1666 [==============================] - 11s 7ms/step - loss: 7.4736
Epoch 3/30
1666/1666 [==============================] - 10s 6ms/step - loss: 7.2826
Epoch 4/30
1666/1666 [==============================] - 11s 7ms/step - loss: 7.0797
Epoch 5/30
1666/1666 [==============================] - 14s 8ms/step - loss: 6.8530
Epoch 6/30
1666/1666 [==============================] - 14s 8ms/step - loss: 6.6070
Epoch 7/30
1666/1666 [==============================] - 11s 7ms/step - loss: 6.3579
Epoch 8/30
1666/1666 [==============================] - 11s 7ms/step - loss: 6.1128
Epoch 9/30
1666/1666 [==============================] - 11s 6ms/step - loss: 5.8745
Epoch 10/30
1666/1666 [==============================] - 10s 6ms/step - loss: 5.6472
Epoch 11/30
1666/1666 [==============================] - 11s 7ms/step - loss: 5.4292
Epoch 12/30
1666/1666 [==============================] - 11s 6ms/step - l

In [ ]:
model.save('drive/MyDrive/00AI/RNN_model')